<a href="https://colab.research.google.com/github/tiagopecurto/tiagopecurto/blob/main/docs/labs/projs/spbp2425_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas para Processamento de Big Data
## TP1 - Energy Meter Monitoring




The sensor data corresponds to regular readings from 11 residential energy meters. The data covers the month of February 2024.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.


## Questions

The following questions should be answered for the month of February and only for this month.

### For the group of sensors:

1. Compute the total energy consumed.

2. Compute the running total energy consumed so far for each day, inclusive.

Note: You can approximate the result but using the last reading of each day from each sensor.

### For each sensor, separately:

3. Compute the total energy consumed and the average energy consumption per day.

4. Compute the day of the month with minimum and maximum energy consumption.

Note: You can approximate the result but using the last reading of each day from each sensor.

### For each sensor, separately, with estimations:

**Assumptions:**

+ Readings may be missing for extended periods due to communication problems with the sensors.

+ Readings are collected do not fall precisely "on the hour". The are collected and recorded any time.

+ For more precise results, estimate the value of the meter at precise timestamp, using linear interpolation from nearest readings.

5. Compute the **estimated** value of each sensor meter for every hour and day of the month (in ascending order).

6. Compute the **estimated** running total of the energy consumed so far. The value should be updated every hour.

## Requeriments

Solve each question using Structured Spark, either Dataframes or SQL or both.

## Other Grading Criteria

+ Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline
+ November 10, 23h59

For each day late, ***0.5 / day penalty***. Penalty accumulates until the grade of the assignment reaches 8.0.

---
### Colab Setup


In [ ]:
#@title Install PySpark
!pip install pyspark findspark --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
#@title Download the dataset

!wget -q -O energy-readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!head -2 energy-readings.csv

date;sensor;energy
2024-02-01 00:00:00;D;2615.0


In [ ]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('energy').getOrCreate()

sc = spark.sparkContext
try :
    readings = spark.read.csv('energy-readings.csv',
                             sep =';', header=True, inferSchema=True)

    readings.printSchema()

    #0.Filtra os dados para manter apenas o mês de fevereiro de 2024
    readings_february = readings.filter((month("date") == 2) & (year("date") == 2024))
      #readings_february.show()

    #1.Total energy consumed in February
    total_energy_consumed = readings_february.groupBy("sensor") \
      .agg((max("energy")-min("energy")).alias("energy_consumed_per_sensor")) \
      .agg(sum("energy_consumed_per_sensor").alias("total_energy_consumed"))
      #total_energy_consumed.show()

    #2.Total energy consumed by day in February
    #2.1: Calcula o consumo diário de cada sensor
    daily_consumption_per_sensor = readings_february.withColumn("day", to_date("date")) \
      .groupBy("day", "sensor") \
      .agg(round((max("energy") - min("energy"))).alias("daily_consumption")) \
      .orderBy("day", "sensor")
    daily_consumption_per_sensor.show()

    #2.2: Soma o consumo diário para todos os sensores por dia
    total_daily_consumption = daily_consumption_per_sensor.groupBy("day") \
      .agg(sum("daily_consumption").alias("total_daily_consumption")) \
      .orderBy("day")
    #total_daily_consumption.show()

    #3.Compute the total energy consumed and the average energy consumption per day
        #Note: utilizou-se a variável de 2.1
        #Note2: a função avg() apenas faz a média com o dias em que houve consumos. é necessário dividir pelos 29 dias
    total_and_average_consumption = daily_consumption_per_sensor.groupBy("sensor") \
      .agg(round(sum("daily_consumption")).alias("total_energy_consumed"),
         (round(sum("daily_consumption")/lit(29))).alias("average_daily_consumption"))\
      .orderBy("sensor")
    #total_and_average_consumption.show()

    #4.Compute the day of the month with minimum and maximum energy consumption
    min_consumption_day = daily_consumption_per_sensor.groupBy("sensor") \
    .agg(min("daily_consumption").alias("min_daily_consumption"))
    min_consumption_day.show()
    """
    min_day = daily_consumption_per_sensor.join(min_consumption_day(daily_consumption_per_sensor["sensor"] == min_consumption_day["sensor"]) &
    (daily_consumption_per_sensor["daily_consumption"] == min_consumption_day["min_daily_consumption"]),"inner") \
    .select(daily_consumption_per_sensor["sensor"], col("day").alias("min_consumption_day"))
     """



    #readings.show(5)
except Exception as err:
    print(err)

**Question 1** - For the group of sensors compute the total energy consumed

In [27]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('energy').getOrCreate()

sc = spark.sparkContext
try :
    readings = spark.read.csv('energy-readings.csv',
                             sep =';', header=True, inferSchema=True)

    #0.Filtra os dados para manter apenas o mês de fevereiro de 2024
    readings_february = readings.filter((month("date") == 2) & (year("date") == 2024))
      #readings_february.show()

    #1.Total energy consumed in February
    total_energy_consumed = readings_february.groupBy("sensor") \
      .agg((max("energy")-min("energy")).alias("energy_consumed_per_sensor")) \
      .agg(round(sum("energy_consumed_per_sensor"),3).alias("Total Energy Consumed"))

    total_energy_consumed.show()

except Exception as err:
    print(err)

+---------------------+
|Total Energy Consumed|
+---------------------+
|              3106.81|
+---------------------+



**Question 2** - For the group of sensors, compute the running total energy consumed so far for each day, inclusive

In [29]:
try :
  #2.Total energy consumed by day in February
  #2.1: Calcula o consumo diário de cada sensor
  daily_consumption_per_sensor = readings_february.withColumn("day", to_date("date")) \
    .groupBy("day", "sensor") \
    .agg(round(max("energy") - min("energy"),3).alias("daily_consumption")) \
    .orderBy("day", "sensor")
  #daily_consumption_per_sensor.show()

  #2.2: Soma o consumo diário para todos os sensores por dia
  total_daily_consumption = daily_consumption_per_sensor.groupBy("day") \
    .agg(round(sum("daily_consumption"),3).alias("total_daily_consumption")) \
    .orderBy("day")
  total_daily_consumption.show()

except Exception as err:
  print(err)

+----------+-----------------------+
|       day|total_daily_consumption|
+----------+-----------------------+
|2024-02-01|                  119.7|
|2024-02-02|                   69.6|
|2024-02-09|                   55.7|
|2024-02-10|                  113.9|
|2024-02-11|                  117.7|
|2024-02-12|                  110.5|
|2024-02-13|                  112.5|
|2024-02-14|                   92.9|
|2024-02-15|                   81.0|
|2024-02-16|                   47.4|
|2024-02-18|                   57.8|
|2024-02-19|                   79.5|
|2024-02-20|                   84.0|
|2024-02-21|                   83.0|
|2024-02-22|                   76.5|
|2024-02-23|                   84.4|
|2024-02-24|                  63.55|
|2024-02-25|                  99.78|
|2024-02-26|                  92.47|
|2024-02-27|                  92.92|
+----------+-----------------------+
only showing top 20 rows



**Question 3** - For each sensor separately, compute the total energy consumed and the average energy consumption per day.

In [30]:
#3.Compute the total energy consumed and the average energy consumption per day
        #Note: utilizou-se a variável de 2.1
        #Note2: a função avg() apenas faz a média com o dias em que houve consumos. é necessário dividir pelos 29 dias
try :
  total_and_average_consumption = daily_consumption_per_sensor.groupBy("sensor") \
        .agg(round(sum("daily_consumption"),3).alias("total_energy_consumed"),
          (round(sum("daily_consumption")/lit(29),3)).alias("average_daily_consumption"))\
        .orderBy("sensor")

  total_and_average_consumption.show()

except Exception as err:
  print(err)

+------+---------------------+-------------------------+
|sensor|total_energy_consumed|average_daily_consumption|
+------+---------------------+-------------------------+
|     A|               113.18|                    3.903|
|     B|                73.51|                    2.535|
|     C|               175.21|                    6.042|
|     D|                341.7|                   11.783|
|     E|               286.59|                    9.882|
|     F|                 97.1|                    3.348|
|     G|               105.87|                    3.651|
|     H|               290.88|                    10.03|
|     I|                206.7|                    7.128|
|     J|               141.81|                     4.89|
|     K|               148.05|                    5.105|
+------+---------------------+-------------------------+



**Question 4** - For each sensor separately, compute the day of the month with minimum and maximum energy consumption.

In [33]:

# 4.1 Calcula o valor mínimo e máximo de consumo diário para cada sensor
min_consumption_day = daily_consumption_per_sensor.groupBy("sensor") \
    .agg(min("daily_consumption").alias("min_daily_consumption"))

max_consumption_day = daily_consumption_per_sensor.groupBy("sensor") \
    .agg(max("daily_consumption").alias("max_daily_consumption"))

# 4.3 Filtra para encontrar o dia onde o consumo foi mínimo
# Adiciona aliases para resolver a ambiguidade
min_day = daily_consumption_per_sensor.alias("dcps").join(
    min_consumption_day.alias("min_day"),
    (col("dcps.sensor") == col("min_day.sensor")) &
    (col("dcps.daily_consumption") == col("min_day.min_daily_consumption")),"inner"
).select(
    col("dcps.sensor"),
    col("dcps.day").alias("min_consumption_day"),
    col("dcps.daily_consumption").alias("min_daily_consumption")
).orderBy("sensor")

# 4.4 Filtra para encontrar o dia onde o consumo foi máximo
max_day = daily_consumption_per_sensor.alias("dcps").join(
    max_consumption_day.alias("max_day"),
    (col("dcps.sensor") == col("max_day.sensor")) &
    (col("dcps.daily_consumption") == col("max_day.max_daily_consumption")),"inner"
).select(
    col("dcps.sensor"),
    col("dcps.day").alias("max_consumption_day"),
    col("dcps.daily_consumption").alias("max_daily_consumption")
).orderBy("sensor")



# 4.5 Exibe os resultados
min_day.show()
max_day.show()

+------+-------------------+---------------------+
|sensor|min_consumption_day|min_daily_consumption|
+------+-------------------+---------------------+
|     A|         2024-02-24|                 0.77|
|     B|         2024-02-09|                  0.1|
|     C|         2024-02-02|                  1.6|
|     C|         2024-02-23|                  1.6|
|     D|         2024-02-16|                  5.7|
|     E|         2024-02-23|                  4.7|
|     F|         2024-02-18|                  0.8|
|     G|         2024-02-09|                  0.7|
|     H|         2024-02-26|                  2.1|
|     I|         2024-02-18|                  0.5|
|     J|         2024-02-18|                  1.7|
|     K|         2024-02-16|                  1.2|
+------+-------------------+---------------------+

+------+-------------------+---------------------+
|sensor|max_consumption_day|max_daily_consumption|
+------+-------------------+---------------------+
|     A|         2024-02-01|  

In [47]:
from pyspark.sql import Window
from pyspark.sql.functions import col, lag, lead, to_date, unix_timestamp, expr, sequence, explode, lit
from pyspark.sql.types import DateType
from datetime import datetime, timedelta

# Passo 1: Extrair o valor máximo ou mínimo diário para cada sensor
daily_extreme = readings_february.withColumn("day", to_date("date")) \
    .groupBy("sensor", "day") \
    .agg(max("energy").alias("daily_extreme_energy")) \
    .orderBy("sensor", "day")

# Passo 2: Gerar um DataFrame com todas as datas de fevereiro para cada sensor usando datetime
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 29)
all_days = [(start_date + timedelta(days=i)).date() for i in range((end_date - start_date).days + 1)]

# Cria um DataFrame com todas as datas para fevereiro
all_days_df = spark.createDataFrame([(d,) for d in all_days], ["day"])

# Obter os sensores distintos
sensors_df = readings_february.select("sensor").distinct()

# Realizar um cross join entre os sensores e todas as datas para obter todas as combinações
all_days_per_sensor = sensors_df.crossJoin(all_days_df)

# Passo 3: Fazer um left join para garantir que todos os dias aparecem para cada sensor
full_daily_data = all_days_per_sensor.join(daily_extreme, ["sensor", "day"], "left") \
    .orderBy("sensor", "day")

# Passo 4: Definir a janela e aplicar a interpolação linear entre o último e o próximo valor conhecido para blocos
window_spec = Window.partitionBy("sensor").orderBy("day")

# Adicionar colunas com as leituras anteriores e seguintes para interpolação
# Interpolação linear para preencher blocos de dias consecutivos NULL
interpolated_data = full_daily_data \
    .withColumn("prev_day", lag("day").over(window_spec)) \
    .withColumn("next_day", lead("day").over(window_spec)) \
    .withColumn("prev_energy", lag("daily_extreme_energy").over(window_spec)) \
    .withColumn("next_energy", lead("daily_extreme_energy").over(window_spec)) \
    .withColumn("interpolated_energy", expr("""
        CASE
            WHEN daily_extreme_energy IS NOT NULL THEN daily_extreme_energy
            WHEN prev_energy IS NOT NULL AND next_energy IS NOT NULL THEN
                prev_energy + (unix_timestamp(day) - unix_timestamp(prev_day)) /
                (unix_timestamp(next_day) - unix_timestamp(prev_day)) *
                (next_energy - prev_energy)
            ELSE NULL
        END
    """))

# Selecionar o resultado final com o valor interpolado para blocos
result = interpolated_data.select("sensor", "day", "daily_extreme_energy", "interpolated_energy") \
    .orderBy("sensor", "day")

# Exibir os resultados interpolados
result.show(50)

+------+----------+--------------------+-------------------+
|sensor|       day|daily_extreme_energy|interpolated_energy|
+------+----------+--------------------+-------------------+
|     A|2024-02-01|               658.6|              658.6|
|     A|2024-02-02|               663.4|              663.4|
|     A|2024-02-03|                NULL|               NULL|
|     A|2024-02-04|                NULL|               NULL|
|     A|2024-02-05|                NULL|               NULL|
|     A|2024-02-06|                NULL|               NULL|
|     A|2024-02-07|                NULL|               NULL|
|     A|2024-02-08|                NULL|               NULL|
|     A|2024-02-09|               712.5|              712.5|
|     A|2024-02-10|               719.9|              719.9|
|     A|2024-02-11|               726.3|              726.3|
|     A|2024-02-12|               733.0|              733.0|
|     A|2024-02-13|               739.8|              739.8|
|     A|2024-02-14|     